In [1]:
%load_ext autoreload
%autoreload 2

DATA PROCESSING 

In [2]:
from Bio import SeqIO
from Bio.PDB.PDBParser import PDBParser
from pathlib import Path
import numpy as np
import pandas as pd
import torch

In [3]:
basedir = Path('.').resolve()
pdb_file_path = (basedir / 'data/rna/7K00_23S.pdb').as_posix()
parser = PDBParser(PERMISSIVE=1)
struc = parser.get_structure('23S', pdb_file_path)

wt_seq = SeqIO.read(basedir / 'data/rna/7K00_sequence.fasta', 'fasta')
wt_seq = str(wt_seq.seq)

wt_seq_match = np.asarray([nt for nt in wt_seq if nt == nt.upper()])
res_idx_match = np.asarray([i for i,nt in enumerate(wt_seq) if nt == nt.upper()])
# mask is for Ecoli match states (i.e. match states that are not deleted in the E-coli sequence)
mask = np.asarray([(x != x.lower()) and (x != '-') for x in wt_seq_match])

dist_map = pd.read_csv(basedir / 'data/rna/7K00_distance_map.csv', header=None).values[mask, :][:, mask]
dist_map = torch.tensor(dist_map)
np.save(basedir / 'data/rna/processed_for_ml' / 'distance_map.npy', dist_map.cpu().numpy())

## Train set - 90/10 split for early stopping

In [5]:
allowed_chars = {'A', 'U', 'C', 'G', '-'}
MSA = [str(s.seq) for s in SeqIO.parse(basedir / 'data/rna/processed_for_ml/feb_24_23s_final_splits/final_23S_GNN_Train.afa', 'fasta')]
MSA = [np.asarray(list(x))[mask].tolist() for x in MSA]
MSA = [''.join([nt if nt in allowed_chars else 'X' for nt in seq]) for seq in MSA]

with open(basedir / 'data/rna/processed_for_ml/feb_24_23s_final_splits' / '23S_alignment.txt', 'w') as outfn:
    for s in MSA:
        outfn.write(f'{s}\n')

In [6]:
basedir = Path('.').resolve()
processed_dir = basedir / 'data/rna/processed_for_ml/feb_24_23s_final_splits'


i_to_nt = ['A', 'U', 'C', 'G', '-', 'X']
nt_to_i = {nt:i for i, nt in enumerate(i_to_nt)}
MSA = [s.strip() for s in open(processed_dir / '23S_alignment.txt')]
MSA_enc = torch.tensor([[nt_to_i[nt] for nt in s] for s in MSA]).cpu()
idxs = torch.randperm(len(MSA_enc), device='cpu')

In [7]:
num_train = int(0.9*len(MSA_enc)) # 90% train, 10% val

train_idxs = idxs[:num_train]
val_idxs = idxs[num_train:]

torch.save(MSA_enc[train_idxs], processed_dir / 'train.pt')
torch.save(MSA_enc[val_idxs], processed_dir / 'val.pt')

## Test set

In [10]:
allowed_chars = {'A', 'U', 'C', 'G', '-'}
MSA = [str(s.seq) for s in SeqIO.parse(basedir / 'data/rna/processed_for_ml/feb_24_23s_final_splits/final_23S_GNN_Test.afa', 'fasta')]
MSA = [np.asarray(list(x))[mask].tolist() for x in MSA]
MSA = [''.join([nt if nt in allowed_chars else 'X' for nt in seq]) for seq in MSA]

with open(basedir / 'data/rna/processed_for_ml/feb_24_23s_final_splits' / '23S_alignment_test.txt', 'w') as outfn:
    for s in MSA:
        outfn.write(f'{s}\n')




In [11]:
basedir = Path('.').resolve()
processed_dir = basedir / 'data/rna/processed_for_ml/feb_24_23s_final_splits'


i_to_nt = ['A', 'U', 'C', 'G', '-', 'X']
nt_to_i = {nt:i for i, nt in enumerate(i_to_nt)}
MSA = [s.strip() for s in open(processed_dir / '23S_alignment_test.txt')]
MSA_enc = torch.tensor([[nt_to_i[nt] for nt in s] for s in MSA]).cpu()
idxs = torch.randperm(len(MSA_enc), device='cpu')

In [12]:
torch.save(MSA_enc, processed_dir / 'test.pt')
